<img src="../common/rfsoc_book_banner.jpg" alt="University of Strathclyde" align="left">

# Notebook Set B

---

## 02 - Quantisation
In this notebook we will investigate quantisation, which is the process of mapping an analogue amplitude acquired by an ADC to a discrete set of values. We will explore the quantisation process and discuss resolution and discrete levels. We will briefly discuss the total achievable dynamic range for a given $N$-bit converter and then investigate errors and noise that may occur during quantisation.

## Table of Contents
* [1. Introduction](#introduction)
* [2. Quantisation](#quantisation)
    * [2.1. Resolution and Levels](#resolution-and-levels)
    * [2.2. Dynamic Range](#dynamic-range)
    * [2.3. Noise](#noise)
* [3. Conclusion](#conclusion)

## Revision
* **v1.0** | 21/01/23 | *First Revision*

---


## 1. Introduction <a class="anchor" id="introduction"></a>
Before we explore quantisation, let us begin by generating a test waveform. We are unable to generate a truly continuous waveform to be discretely sampled and quantised. This is because we are working on a digital system that is unable to generate continuous waves. Instead, we will use a high sampling rate to approximate a continuous wave.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import rfsoc_book.helper_functions as hf

fs_cont = 10e3 #Hz
ts_cont = 1/fs_cont
n_cont  = np.arange(0, 2000)
fd = 13 #Hz

x_cont = n_cont/fs_cont
y_cont = np.sin(2*np.pi*fd*ts_cont*n_cont)

hf.plot_timeseries("Continuous Waveform",
                   [x_cont], [y_cont],
                   ['continuous'])

## 2. Quantisation <a class="anchor" id="quantisation"></a>
We investigated the sampling process in the [previous notebook](01_sampling.ipynb), where an analogue waveform was converted to a digital waveform using a regular sampling period. Quantisation is the process of mapping each sample of the digital wave to a discrete set of possible amplitude levels.

### 2.1. Resolution and Levels <a class="anchor" id="resolution-and-levels"></a>
An ADC performs quantisation using $N$ bits of resolution, which means it can represent $2^N$ different possible values. A linear quantiser (shown in Figure 1) maps the samples of an input wave to a discrete set of quantisation levels.

<figure>
<img src="./images/linear_quantisers.png" style="width: 50%; vertical-align: middle;"/>
    <figcaption><b>Figure 1: A plot demonstrating the functionality of a linear quantiser.</b></figcaption>
</figure>

We can obtain the quantisation step $q$ of an ADC as

$$
q = \frac{V_{\text{max}}}{2^{N-1}}.
$$

For a 4 bit ADC, with $V_\text{max} = 1$, the quantisation levels are given below.

In [ ]:
Vmax = 1
N  = 4
q  = Vmax/(2**(N-1))
levels = np.arange(-1, 1, q)

print('Quantisation levels for 4 bits:')
for idx, level in enumerate(levels):
    print('Decimal: ', idx-2**(N-1), ' |  Discrete Level: ', level)

We can discretely sample the continuous waveform from before without quantisation, as shown in the following code cell.

In [ ]:
# Set sampling parameters
fs = 100 #Hz
ts = 1/fs
n  = np.arange(0, 21)

# Perform sampling process
x = n/fs
y = np.sin(2*np.pi*fd*ts*n)

# And plot
hf.plot_timeseries("Discrete Signal Sampled at 100Hz, No Amplitude Quantisation",
                   [x_cont, x], [y_cont, y],
                   ['continuous', 'discrete'])

The wave can be quantised using 4 bits of resolution, which results in the plot given below.

In [ ]:
# Perform quantisation (4 bits)
yq4 = np.zeros(np.size(y))
N  = 4
q4  = 1/(2**(N-1))
levels = np.arange(-1, 1, q4)
for idx, sample in enumerate(y):
    # https://stackoverflow.com/questions/12141150
    yq4[idx] = min(levels, key=lambda x:abs(x-sample))

hf.plot_timeseries("Quantisation using 4 Bits",
                   [x_cont, x], [y_cont, yq4],
                   ['continuous', 'discrete'])

Notice that many of the sample points do not lie directly on the original continuous waveform. Increasing the resolution to 6 bits improves the quantisation step (and accuracy) of the quantisation process, as shown below.

In [ ]:
# Perform quantisation (6 bits)
yq6 = np.zeros(np.size(y))
N  = 6
q6  = 1/(2**(N-1))
levels = np.arange(-1, 1, q6)
for idx, sample in enumerate(y):
    yq6[idx] = min(levels, key=lambda x:abs(x-sample))

hf.plot_timeseries("Quantisation using 6 Bits",
                   [x_cont, x], [y_cont, yq6],
                   ['continuous', 'discrete'])

### 2.2. Dynamic Range <a class="anchor" id="dynamic-range"></a>
We can compute the Dynamic Range of an $N$-bit converter as follows.

$$
\text{Dynamic Range} = 20\text{log}_{10}(2^N)=20N\text{log}_{10}(2)=6.02N
$$

Therefore, the dynamic range of an ADC should increase as the number of bits increase. We can demonstrate this relationship in the following code cell.

In [ ]:
N_vec = np.arange(2, 17, 1)
DR_vec = N_vec * 6.02
for idx, DR in enumerate(DR_vec):
    print('Number of bits: ', N_vec[idx], ' | Dynamic Range (dB): ', np.round(DR))

### 2.3. Noise <a class="anchor" id="noise"></a>
Quantisation noise can be modelled as Additive White Gaussian Noise (AWGN). All quantised samples are subject to an error. The range of possible error values are in the range $-q/2$ to $q/2$ if we assume the input signal to the quantiser is random. A Probability Density Function (PDF) denoted as $p(e)$ can be formed as given in Figure 2.

<figure>
<img src="./images/q_error_PDF.png" style="width: 25%; vertical-align: middle;"/>
    <figcaption><b>Figure 2: A plot of the PDF of quantisation noise for the quantisation interval $q$.</b></figcaption>
</figure>

A technique known as dithering can be used to break-up periodic noise components in a quantised signal. Before we implement dithering, we will first generate a test waveform.

In [ ]:
# Set sampling parameters
fs = 100 #Hz
ts = 1/fs
L  = 2048
n  = np.arange(0, L)
fd = 10.001

# Perform sampling process
x = n/fs
y = 0.7071*np.sin(2*np.pi*fd*ts*n)

# And plot
hf.plot_timeseries("Discrete Signal Sampled at 100Hz, No Amplitude Quantisation",
                   [x[0:100]], [y[0:100]],
                   ['discrete'])

The log-scale power spectra of this waveform can be inspected below. Notice that the primary lobe is over the desired sine wave frequency of 10Hz. There are no other significant tones in the sampled wave.

In [ ]:
Y = np.fft.fft(y)
Y_norm = np.abs(Y)*2/L
Y_log = 20*np.log10(Y_norm)

fig = plt.figure(figsize=(10,4))
axes = fig.add_subplot(1, 1, 1)
axes.plot(n[0:L//2+1]*fs/L, Y_log[0:L//2+1])
axes.set_title('Power Spectra of the Sampled Wave')
axes.set_xlabel('Frequency (Hz)')
axes.set_ylabel('Power Spectra (dB)')
axes.grid(True, 'Major')

We will now quantise the waveform above using 4 bits of precision.

In [ ]:
yq4 = np.zeros(np.size(y))
N   = 4
q4  = 1/(2**(N-1))
levels = np.arange(-1, 1, q4)
for idx, sample in enumerate(y):
    # https://stackoverflow.com/questions/12141150
    yq4[idx] = min(levels, key=lambda x:abs(x-sample))
    
hf.plot_timeseries("Quantisation using 4 Bits",
                   [x[0:100]], [yq4[0:100]],
                   ['discrete'])

Errors have been introduced due to quantisation. We can easily inspect spurious tones introduced by quantisation by plotting the power spectra of the quantised wave.

In [ ]:
import matplotlib.pyplot as plt

Yq4 = np.fft.fft(yq4)
Yq4_norm = np.abs(Yq4)*2/L
Yq4_log = 20*np.log10(Yq4_norm)

fig = plt.figure(figsize=(10,4))
axes = fig.add_subplot(1, 1, 1)
axes.plot(n[0:L//2+1]*fs/L, Yq4_log[0:L//2+1])
axes.set_title('Power Spectra of the Quantised Wave')
axes.set_xlabel('Frequency (Hz)')
axes.set_ylabel('Power Spectra (dB)')
axes.grid(True, 'Major')

Notice that the power spectra of the quantised wave contains tones that were not in the original sampled wave.

We can derive the power of the quantisation error. The square of the error signal, weighted by the probability of error, is integrated across all possible error values. We can perform a definite integral as the range of possible error values is between $-q/2$ and $q/2$.

$$
n_{\text{ADC}} = \int_{-\inf}^{\inf}e^2p(e)de = \int_{-q/2}^{q/2}e^2p(e)de
$$

Since $p(e)=1/q$ for all values of $e$, the error power is

$$
n_{\text{ADC}} = \frac{1}{3q}e^3 \Big|_{-q/2}^{q/2} = \frac{q^2}{12}.
$$

We can compute the noise power below.

In [ ]:
noise_power = (q4**2)/12
noise_power

A normal (Gaussian) distribution can be used to generate a dither signal for the given noise power above. For a Gaussian random variable, the average power is calculated by summing the mean value and the variance, which is also known as the second moment.

$$
\text{average power} = \mu^2 + \sigma^2
$$

The expected mean value of our dither signal is $\mu = 0$. Therefore, the average power now becomes,

$$
\text{average power} = \sigma^2.
$$

The average power of our dither signal will be $n_{\text{ADC}}$. Now we obtain,

$$
n_{\text{ADC}} = \sigma^2.
$$

We can generate a normal distribution to act as our dither signal. The Numpy random module has a normal distribution method that accepts a mean value and standard deviation $\sigma$. We can obtain the standard deviation by simply computing the square root of the average power, as below.

$$
\sqrt{n_{\text{ADC}}} = \sigma.
$$

Now, running the code cell below, we generate our dither signal using `np.random.normal(mean, standard deviation, length)`.

In [ ]:
dither = np.random.normal(0,np.sqrt(noise_power),L)

hf.plot_timeseries("Dither Signal",
                   [x[0:100]], [dither[0:100]],
                   ['discrete'])

The dither signal is added to the original sampled signal before it is quantised, as below.

In [ ]:
yd = y+dither

yq4d = np.zeros(np.size(y))
for idx, sample in enumerate(yd):
    # https://stackoverflow.com/questions/12141150
    yq4d[idx] = min(levels, key=lambda x:abs(x-sample))

hf.plot_timeseries("Quantised Signal with added Noise",
                   [x[0:100]], [yq4d[0:100]],
                   ['discrete'])

We can observe the spectrum of the quantised signal with dither and overlay it on the same plot as the original quantised signal.

In [ ]:
Yq4d = np.fft.fft(yq4d)
Yq4d_norm = np.abs(Yq4d)*2/L
Yq4d_log = 20*np.log10(Yq4d_norm)

fig = plt.figure(figsize=(10,4))
axes = fig.add_subplot(1, 1, 1)
axes.plot(n[0:L//2+1]*fs/L, Yq4_log[0:L//2+1])
axes.plot(n[0:L//2+1]*fs/L, Yq4d_log[0:L//2+1])
axes.set_title('Comparing Power Spectra with and without Dithering')
axes.set_xlabel('Frequency (Hz)')
axes.set_ylabel('Power Spectra (dB)')
axes.grid(True, 'Major')

Notice that dithering has broken-up periodic elements in the quantised signal, causing the spurious tones to be suppressed.

## 3. Conclusion
This notebook explored the quantisation process and has introduce dithering.

The next notebook investigates ADCs and DACs.

---

[⬅️ Previous Notebook](01_sampling.ipynb) || [Next Notebook 🚀](03_adcs_and_dacs.ipynb)

Copyright © 2023 Strathclyde Academic Media

---
---